# Project Final Report

## Introduction

**Background**  
In recent years, online gaming has produced vast behavioral datasets that offer insights into user engagement and retention. “Minecraft,” as an open‐world game, generates detailed logs of player activity, making it an ideal case for studying how simple demographic factors relate to play behavior. By analyzing total play duration alongside basic player attributes, we aim to uncover patterns that could inform game design and personalized recommendations.

**Research Question**  
Can a player’s age and total play time predict their experience level on the game server?

- **Response variable**: `experience` (factor) – player’s self‐reported expertise level (“Pro”, “Regular”, “Amateur”, “Beginner”)  
- **Predictor variables**:  
  - `Age` (numeric; years)  
  - `played_hours` (numeric; total play duration in hours)  

**Data Description**  
We use the `players.csv` dataset exported from the research server. It contains **196** observations and **9** variables:

| variable           | type       | description                                    |
|--------------------|------------|------------------------------------------------|
| experience         | factor     | player’s experience level                      |
| subscribe          | logical    | whether the player holds a subscription        |
| hashedEmail        | character  | anonymized user identifier                     |
| played_hours       | double     | total play duration per player (hours)         |
| name               | character  | player’s display name                          |
| gender             | character  | player’s self‐reported gender                  |
| age                | double     | player’s age in years                          |
| individualId       | logical    | all NA (no values) — will be dropped           |
| organizationName   | logical    | all NA (no values) — will be dropped           |

For this analysis, we will:

1. Convert `experience` to a factor.  
2. Drop the two all‐NA columns (`individualId`, `organizationName`).  
3. Focus only on the three variables needed for prediction: `experience`, `Age`, and `played_hours`.  




## Methods & Results  
In this phase, we apply a classification framework because our response variable, `experience`, is inherently categorical rather than numerical. By first recasting the character field `experience` into a factor, we can then use classification algorithms to predict it. We posit that a player’s `age` and total `played_hours` are the key features driving their experience level, so these two variables will form the predictors in our model.

## Load & wrangle  
We start by loading the R packages needed for data cleaning and modeling. Using `read_csv()`, we import the `players.csv` dataset and inspect its top rows with `head()`. Next, we transform `experience` from a text column into a factor suitable for classification, and then pare down the data to the three fields—`experience`, `Age`, and `played_hours`—that we will use in the subsequent analysis.  


In [3]:
# Load required packages for data manipulation and modeling
library(tidyverse)   
library(tidymodels)   

# Read in the players.csv file and peek at the first 10 rows
players <- read_csv("players.csv")
head(players, 10)

# Convert the response variable 'experience' from character to factor,
# and keep only the three columns we will use in the classification
ready_players <- players |>
  mutate(experience = as_factor(experience)) |>
  select(experience, Age, played_hours)

# Show the cleaned dataset
ready_players


Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17
Regular,TRUE,8e594b8953193b26f498db95a508b03c6fe1c24bb5251d392c18a0da9a722807,0.0,Luna,Female,19
Amateur,FALSE,1d2371d8a35c8831034b25bda8764539ab7db0f63938696917c447128a2540dd,0.0,Emerson,Male,21
Amateur,TRUE,8b71f4d66a38389b7528bb38ba6eb71157733df7d1740371852a797ae97d82d1,0.1,Natalie,Male,17


experience,Age,played_hours
<fct>,<dbl>,<dbl>
Pro,9,30.3
Veteran,17,3.8
Veteran,17,0.0
Amateur,21,0.7
Regular,21,0.1
Amateur,17,0.0
Regular,19,0.0
Amateur,21,0.0
Amateur,17,0.1


In [2]:
players <- read_csv("players.csv")
head(players)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17
